In [1]:
from fipy import CellVariable, Grid1D, TransientTerm, ConvectionTerm, DiffusionTerm
from fipy.tools import numerix
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Parameters
nu = 0.01/np.pi # Kinematic viscosity
L = 2.0    # Spatial domain length (-1 to 1)
N = 1000    # Number of spatial points
dx = L / N  # Spatial step size
x = np.linspace(-1, 1, N)  # Spatial grid

# Mesh
mesh = Grid1D(dx=dx, nx=N) - 1

# Variable to solve for
u = CellVariable(name="u", mesh=mesh, value=0.0)

# Initial condition
u.setValue(-np.sin(np.pi * mesh.cellCenters[0]))

# Boundary conditions
u.constrain(0, where=mesh.facesLeft)   # u(t, -1) = 0
u.constrain(0, where=mesh.facesRight)  # u(t, 1) = 0

# Define the PDE
eq = (TransientTerm() + ConvectionTerm(coeff=u.faceValue * [[1]]) - DiffusionTerm(coeff=nu) == 0)

# Time step and total time
dt = 0.001
total_time = 1.0
steps = int(total_time / dt)
time_grid = np.linspace(0, total_time, steps)
u_solution = np.zeros((len(time_grid), len(mesh.cellCenters[0])))


for step in (range(steps)):
    eq.solve(var=u, dt=dt)
    u_solution[step, :] = u.value

    if step % 250 == 0:
        plt.plot(mesh.cellCenters[0], u.value, label=f't={step * dt:.2f}')
plt.plot(mesh.cellCenters[0], u.value, label=f't={step * dt:.2f}')

# Final plot
plt.xlabel('x')
plt.ylabel('u(t, x)')
plt.legend()
plt.grid()
plt.show()

In [ ]:
T, X = np.meshgrid(time_grid, mesh.cellCenters[0], indexing='ij')

# Plot the 2D contour
plt.figure(figsize=(10, 4))
contour = plt.contourf(T, X, u_solution, levels=100, cmap='RdBu_r')
plt.colorbar(contour, label='u(t, x)')
plt.xlabel('t')
plt.ylabel('x')
plt.show()